# **Alertas de Conformidade**

**Alertas de conformidade** são notificações geradas a partir de violações políticas ou regulamentos de uma organização.

## **Dicionário de Dados**

| Campo                     | Tipo     | Descrição                                                                 |
|:--------------------------|:---------|:--------------------------------------------------------------------------|
| **alert_id**              | string   | Identificador único do alerta.                                            |
| **creation_datetime**     | datetime | Data e hora em que o alerta foi gerado.                                   |
| **impact_level**          | string   | Nível de impacto para a organização/equipe.                               |
| **status**                | string   | Status do alerta.                                                         |
| **assigned_to**           | string   | Time/equipe responsável.                                                  |
| **type_of_alert**         | string   | Tipo do alerta.                                                           |
| **resolutation_datetime** | datetime | Data e hora da resolução/finalização do alerta.                           |
| **conclusion**            | string   | Conclusão da análise.                                                     |

### **1. Varredura e Extração de Dados de uma API**

##### **1.1. Bibliotecas utilizadas:**
* *flask*: Microframework web. Utilizado para simular a API.
  
* *threading*: Cria e controla threads (unidades de execução paralelas). Biblioteca utilizada para conseguir executar as requisições, da simulação da API, desenvolvida com o microframework *flask*. O código foi desenvolvido via Jupyter Notebook onde já se utiliza uma thread principal para executação do kernel. Portanto, ao executar as requisições, irá bloquear o processo atual impedindo que as outras células sejam executadas. Utilizando o *threading*, o servidor vai rodar em segundo plano, liberando o kernel do notebook para executar as células, evitando assim que o ambiente trave.

* *pandas*: Biblioteca utilizada para trabalhar com os dados de forma tabular.

* *random*: Biblioteca utilizada para gerar dados aleatórios para a criação da base de alertas.

* *datetime*: Biblioteca utilizada para trabalhar com dados de data e hora.

* *uuid*: Biblioteca utilizada para gerar identificadores únicos.

* *csv*: Biblioteca utilizada para escrever a base de alertas em um arquivo delimitado por vírgula (csv). 

In [ ]:
from flask import Flask, jsonify, request
from threading import Thread
import pandas as pd
import random
from datetime import datetime, timedelta
import uuid
import csv

##### **1.2. Mapeamento dos Campos:**
Valores mapeados para cada campo: 
* *alert_id*: Identificador único gerado de forma aleatória.

* *creation_date*: Foi definida uma fórmula para determinar um datetime aleatório do passado, com até 365 dias e 24 horas atrás, a partir do dia atual.

* *impact_level*: Baixo; Médio; Alto; Crítico. 

* *status*: Aberto; Em análise; Concluído. 

* *assigned_to*: Monitoramento, TI, Financeiro, Compliance ou Logística;
  
* *type_of_alert*: Cada equipe possui tipos específicos de alertas, conforme especificado abaixo.

  Monitoramento: Movimentação financeira suspeita; Login suspeito.<br>
  TI: Backup não executado; Registro incompleto no sistema; Atualização de sistema pendente.<br>
  Financeiro: Relatório financeiro com erro; Conformidade fiscal incompleta; Pagamento para fornecedor não homologado.<br>
  Compliance: Conflito de interesse; Código de conduta violado.<br>
  Logística: Entrega fora do prazo; Produto sem rastreamento; Falha no controle de estoque.

* *resolutation_date*: Foi definida uma fórmula para determinar um datetime aleatório (em um intervalo de execução de 7 dias - 168 horas) a partir da data de criação do alerta (*creation_date*), caso o alerta estiver com o status igual a "Concluído". 

* *conclusion*: Positivo; Falso Positivo e Necessita de monitoramento. O campo só irá conter dado se o alerta já possuir uma análise concluída. 

In [ ]:
status = ["Aberto", "Em análise", "Concluído"]

type_of_alert_by_teams = {
    "Monitoramento": [
        "Movimentação financeira suspeita", 
        "Login suspeito"
    ],
    "TI": [
        "Backup não executado",
        "Registro incompleto no sistema",
        "Atualização de sistema pendente"
    ],
    "Financeiro": [
        "Relatório financeiro com erro",
        "Conformidade fiscal incompleta",
        "Pagamento para fornecedor não homologado"
    ],
    "Compliance": [
        "Conflito de interesse",
        "Código de conduta violado"
    ],
    "Logística": [
        "Entrega fora do prazo",
        "Produto sem rastreamento",
        "Falha no controle de estoque"
    ]
}

impact_level = ["Baixo", "Médio", "Alto", "Crítico"]

conclusions = ["Positivo", "Falso positivo", "Necessita monitoramento"]

In [ ]:
alerts = {}
for i in range(1, 201): #Geração de 200 registros.
  alert_id = str(uuid.uuid4()) #uuid4 diz respeito a versão 4 da função. Gera um identificador único de forma aleatória. 
  creation_date = datetime.now() - timedelta(days = random.randint(0, 365), hours = random.randint(0, 24)) #Definindo um datetime aleatório do passado, com até 365 dias e 24 horas atrás, a partir do dia atual.
  assigned_to = random.choice(list(type_of_alert_by_teams.keys())) #Retorna as equipes de forma randômica.
  type_of_alert = random.choice(type_of_alert_by_teams[assigned_to]) #Retorna o tipo de alerta com base na respectiva equipe/time.
  resolved = random.choice([True, False])
  resolutation_date = (creation_date + timedelta(hours = random.randint(0, 168))).strftime("%Y-%m-%d %H:%M:%S") if resolved else None #Definindo um datetime aleatório (em um intervalo de execução de 7 dias - 168 horas) a partir da data de criação do alerta (*creation_date*), caso o alerta estiver com o status igual a "Concluído". 
  #Abaixo, estou atribuindo cada valor para sua respectiva chave dentro do dicionário. 
  alert = {
      "alert_id": alert_id,
      "creation_datetime": creation_date.strftime("%Y-%m-%d %H:%M:%S"),
      "type_of_alert": type_of_alert,
      "impact_level": random.choice(impact_level),
      "status": "Concluído" if resolved else random.choice(["Aberto", "Em análise"]),
      "assigned_to": assigned_to,
      "resolutation_datetime": resolutation_date,
      "conclusion": random.choice(conclusions) if resolved else None
  }
  alerts[alert_id] = alert #Incrementando o dicionário a cada loop. 

##### **1.3. Simulação da API:**

In [ ]:
app = Flask(__name__) #Criação da instância da aplicação Flask. 

In [ ]:
@app.route('/api/alerts', methods = ['GET']) #Definindo a rota que retornará todos os dados da base de alertas. 
def list_alerts(): 
  return jsonify(alerts) #jsonify é uma função do Flask para retornar dados no formato JSON.

In [ ]:
@app.route('/api/alerts/<alert_id>', methods=['GET']) #Definindo rota que retornará dados do ID especificado.
def search_alert(alert_id):
    alert = alerts.get(alert_id)
    #Verifica se o ID especificado está presente no dicionário, se estiver, retornará as respectivas informações, caso contrário exibirá que a mensagem "Alerta não encontrado".
    if alert:
        return jsonify(alert)
    else:
        return jsonify({"erro": "Alerta não encontrado"}), 404

In [ ]:
def run_flask_app():
    app.run(port = 5000) #Definição da porta para a chamada da API.

flask_thread = Thread(target = run_flask_app) #Definindo a Thread paralela para conseguir executar a aplicação do microframework Flask. Importante: Essa etapa foi desenvolvida pois o desenvolvido e teste foram realizados via Jupyter Notebook. 
flask_thread.start()

#http://localhost:5000/api/alerts

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


### **2. Escrita e Desnormalização de Resultados**

##### **2.1. Envio dos resultados para um arquivo .csv:**

In [ ]:
def save_csv(name_file = "alerts.csv"): #Nome do arquivo: "alerts.csv"
    with open(name_file, mode = 'w', newline = '', encoding = 'utf-8') as csvfile:
        fieldnames = ["alert_id", "creation_datetime", "type_of_alert", "impact_level", "status", "assigned_to", "resolutation_datetime", "conclusion"] #Especificando os campos.
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        writer.writeheader()
        for alert in alerts.values():
            writer.writerow(alert) #Incrementando o registro no .csv. 
            
save_csv()

##### **2.2. Desnormalização do JSON:**

In [ ]:
alerts_list = list(alerts.values())
df = pd.json_normalize(alerts_list) #Normalizando o retorno JSON para exibição na forma tabular. 

df

,alert_id,creation_datetime,type_of_alert,impact_level,status,assigned_to,resolutation_datetime,conclusion
0,d6998177-adb1-47e7-8953-04715176ef9c,2024-08-15 19:23:42,Backup não executado,Baixo,Em análise,TI,None,None
1,cfa725d7-43b3-4725-9d14-14530abda5e6,2024-11-06 02:23:42,Código de conduta violado,Alto,Concluído,Compliance,2024-11-09 20:23:42,Necessita monitoramento
2,14091773-ca22-484e-a488-7f2616ce5b03,2025-04-07 01:23:42,Conflito de interesse,Alto,Concluído,Compliance,2025-04-07 04:23:42,Positivo
3,acdb3cd4-a79b-474e-8eb6-040bef6a1925,2025-01-23 05:23:42,Relatório financeiro com erro,Crítico,Concluído,Financeiro,2025-01-26 14:23:42,Necessita monitoramento
4,081b2ea8-094d-435f-85ec-0a08f6638b2b,2024-07-29 23:23:42,Backup não executado,Médio,Concluído,TI,2024-07-31 19:23:42,Necessita monitoramento
...,...,...,...,...,...,...,...,...
195,03158d32-e2c7-49ec-b065-c9d105d84f5a,2024-09-10 21:23:42,Atualização de sistema pendente,Médio,Aberto,TI,None,None
196,df533e82-e48c-46e0-b399-ff31160b0a45,2024-09-19 13:23:42,Conformidade fiscal incompleta,Alto,Concluído,Financeiro,2024-09-20 03:23:42,Positivo
197,b29cec30-b410-46d6-9b8c-0967ca19dab7,2024-11-02 02:23:42,Entrega fora do prazo,Alto,Em análise,Logística,None,None
198,7c60d296-6415-4e7a-9853-9d76ce2146f1,2025-04-22 22:23:42,Pagamento para fornecedor não homologado,Alto,Em análise,Financeiro,None,None
